In [1]:
import pandas as pd
df = pd.read_csv("./all.csv", parse_dates=True)
df

,Unnamed: 0,Date,Text
0,168,2023-04-12,«ментовск войны» обошл приквел «игр престолов»...
1,128,2023-04-12,рпц назва «странной» возможн пасхальн перемир ...
2,174,2023-04-12,«велик отказ» компан росс теря деньг массов фе...
3,172,2023-04-12,набиуллин увидел повод влиян курс рубл цб выст...
4,170,2023-04-12,британ ввел санкц прот usm усманов член сем ке...
...,...,...,...
144344,81842,2015-01-01,cудмедэксперт приступ осмотр тел жертв крушен ...
144345,81843,2015-01-01,кадыр сообщ уничтожен главар бандгрупп мус зав...
144346,81846,2015-01-01,роспотребнадзор призва россия перееда напива п...
144347,81845,2015-01-01,российск спасател присоедин поиск малайзийск с...


In [3]:
import re

class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(word):
        word = word.lower()
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)
        if m is None:
            return word
        if m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp == rv:
                rv = Porter.REFLEXIVE.sub('', rv, 1)
                temp = Porter.ADJECTIVE.sub('', rv, 1)
                if temp != rv:
                    rv = temp
                    rv = Porter.PARTICIPLE.sub('', rv, 1)
                else:
                    temp = Porter.VERB.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.NOUN.sub('', rv, 1)
                    else:
                        rv = temp
            else:
                rv = temp
            
            rv = Porter.I.sub('', rv, 1)

            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)

            temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre+rv
        return word
    stem=staticmethod(stem)
def stemming(s):
    s = s.split()
    s = map(lambda ss: Porter.stem(ss), s)
    return " ".join(s)

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
words = [
    "газпром",
    "газ",
    "нефть",
    "бензин",
    "санкции",
    "кризис",
    "рубль",
    "рост",
    "падение",
    "цена",
    "подорожание",
    "удешевение"
]
_filter = df['Text'].str.contains(" " + stemming(words[0]) + " ")
for w in words:
    s = stemming(w)
    _filter |= df['Text'].str.contains(" " + s + " ")
    _filter |= df['Text'].str.contains("^" + s + " ")
    _filter |= df['Text'].str.contains(" " + s + "$")
df['HasWords'] = _filter
df

,Unnamed: 0,Date,Text,HasWords
0,168,2023-04-12,«ментовск войны» обошл приквел «игр престолов»...,True
1,128,2023-04-12,рпц назва «странной» возможн пасхальн перемир ...,False
2,174,2023-04-12,«велик отказ» компан росс теря деньг массов фе...,False
3,172,2023-04-12,набиуллин увидел повод влиян курс рубл цб выст...,True
4,170,2023-04-12,британ ввел санкц прот usm усманов член сем ке...,True
...,...,...,...,...
144344,81842,2015-01-01,cудмедэксперт приступ осмотр тел жертв крушен ...,False
144345,81843,2015-01-01,кадыр сообщ уничтожен главар бандгрупп мус зав...,False
144346,81846,2015-01-01,роспотребнадзор призва россия перееда напива п...,False
144347,81845,2015-01-01,российск спасател присоедин поиск малайзийск с...,False


In [6]:
funds = pd.read_excel("./gazprom-moscow-exchange.xlsx")
funds['Дата'] = pd.to_datetime(funds['Дата'], dayfirst=True)
funds['Change'] = 0
funds = funds.sort_values("Дата")
for i in range(1, funds.shape[0]):
    funds['Change'].iloc[i] = funds['Цена avg'].iloc[i] - funds['Цена avg'].iloc[i - 1]
funds.tail(10)

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_177908\4236339736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds['Change'].iloc[i] = funds['Цена avg'].iloc[i] - funds['Цена avg'].iloc[i - 1]


,Дата,Bid,Ask,Цена open,Цена min,Цена max,Цена last,Цена avg,Оборот,Кол-во сделок,Change
9,2023-03-28,171.02,171.08,172.94,169.10,172.94,171.07,170.58,4614333299,74028,-1.24
8,2023-03-29,170.91,170.96,171.58,169.41,172.50,170.95,171.13,3841502396,59297,0.55
7,2023-03-30,171.71,171.72,170.78,170.02,171.80,171.71,171.13,2517969423,41954,0.00
6,2023-03-31,169.83,169.85,172.00,168.14,172.71,169.83,169.90,5331859695,89990,-1.23
5,2023-04-03,170.30,170.34,170.99,169.00,171.50,170.38,170.24,3879998447,68103,0.34
4,2023-04-04,172.54,172.55,171.00,170.14,176.00,172.55,173.81,12722343198,149732,3.57
3,2023-04-05,173.86,173.87,172.55,170.58,174.48,173.86,172.95,5691178694,68565,-0.86
2,2023-04-06,171.52,171.56,174.02,171.30,175.00,171.60,173.34,5244430462,78986,0.39
1,2023-04-07,173.07,173.09,171.50,171.29,173.18,173.09,172.30,3089494250,54104,-1.04
0,2023-04-10,174.54,174.55,173.55,173.23,175.20,174.54,174.18,4070633698,64768,1.88


In [8]:
import datetime

changes = {}
for i in range(1, funds.shape[0]):
    start = funds["Дата"].iloc[i - 1]
    end = funds["Дата"].iloc[i]
    date = start
    while date < end:
        changes[str(date)] = funds['Change'].iloc[i]
        date += datetime.timedelta(days=1)
        
df['Change'] = 0
for i in range(df.shape[0]):
    date = df["Date"].iloc[i]
    if str(date) in changes:
        change = changes[str(date)]
        df["Change"].iloc[i] = change
df.head(10)

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_177908\647509515.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Change"].iloc[i] = change


,Unnamed: 0,Date,Text,HasWords,Change
0,168,2023-04-12,«ментовск войны» обошл приквел «игр престолов»...,True,0.0
1,128,2023-04-12,рпц назва «странной» возможн пасхальн перемир ...,False,0.0
2,174,2023-04-12,«велик отказ» компан росс теря деньг массов фе...,False,0.0
3,172,2023-04-12,набиуллин увидел повод влиян курс рубл цб выст...,True,0.0
4,170,2023-04-12,британ ввел санкц прот usm усманов член сем ке...,True,0.0
5,166,2023-04-12,правительств одобр продаж «сахалинск энергии» ...,True,0.0
6,164,2023-04-12,трамп захотел навред сша ответ вопрос кто взор...,False,0.0
7,162,2023-04-12,директор цру заяв неготовн росс переговор укра...,False,0.0
8,160,2023-04-12,ес ответ договор венгр росс заяв «урок зависим...,True,0.0
9,158,2023-04-12,безопасн автомат сдела охран труд эффективн фо...,True,0.0


In [22]:
df['ChangeSign'] = 0
for i in range(df.shape[0]):
    if df['HasWords'].iloc[i]:
        df['ChangeSign'].iloc[i] = 1 if df['Change'].iloc[i] > 0 else 0 if df['Change'].iloc[i] == 0 else -1
df  

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_177908\1282733176.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ChangeSign'].iloc[i] = 1 if df['Change'].iloc[i] > 0 else 0 if df['Change'].iloc[i] == 0 else -1


,Date,Text,HasWords,Change,ChangeSign
0,2023-04-12,«ментовск войны» обошл приквел «игр престолов»...,True,0.0,0
1,2023-04-12,рпц назва «странной» возможн пасхальн перемир ...,False,0.0,0
2,2023-04-12,«велик отказ» компан росс теря деньг массов фе...,False,0.0,0
3,2023-04-12,набиуллин увидел повод влиян курс рубл цб выст...,True,0.0,0
4,2023-04-12,британ ввел санкц прот usm усманов член сем ке...,True,0.0,0
...,...,...,...,...,...
144344,2015-01-01,cудмедэксперт приступ осмотр тел жертв крушен ...,False,0.0,0
144345,2015-01-01,кадыр сообщ уничтожен главар бандгрупп мус зав...,False,0.0,0
144346,2015-01-01,роспотребнадзор призва россия перееда напива п...,False,0.0,0
144347,2015-01-01,российск спасател присоедин поиск малайзийск с...,False,0.0,0


In [20]:
df = df.drop(columns=["Unnamed: 0"])
df

,Date,Text,HasWords,Change,ChangeSign
0,2023-04-12,«ментовск войны» обошл приквел «игр престолов»...,True,0.0,-1
1,2023-04-12,рпц назва «странной» возможн пасхальн перемир ...,False,0.0,0
2,2023-04-12,«велик отказ» компан росс теря деньг массов фе...,False,0.0,0
3,2023-04-12,набиуллин увидел повод влиян курс рубл цб выст...,True,0.0,-1
4,2023-04-12,британ ввел санкц прот usm усманов член сем ке...,True,0.0,-1
...,...,...,...,...,...
144344,2015-01-01,cудмедэксперт приступ осмотр тел жертв крушен ...,False,0.0,0
144345,2015-01-01,кадыр сообщ уничтожен главар бандгрупп мус зав...,False,0.0,0
144346,2015-01-01,роспотребнадзор призва россия перееда напива п...,False,0.0,0
144347,2015-01-01,российск спасател присоедин поиск малайзийск с...,False,0.0,0


In [23]:
df.to_csv('./signed.csv', encoding='utf-8-sig')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 451: character maps to <undefined>